<a href="https://colab.research.google.com/github/tran-linh41214/Python/blob/main/Tr%E1%BA%A7n_Di%E1%BB%87u_Linh_data_wrangling_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
payment_report = pd.read_csv('/content/drive/My Drive/DAC_Python_Project 2/payment_report.csv')
product = pd.read_csv('/content/drive/My Drive/DAC_Python_Project 2/product.csv')
transactions = pd.read_csv('/content/drive/My Drive/DAC_Python_Project 2/transactions.csv')

# **Part I: EDA**

Merge payment_report và product

In [ ]:
payment_enriched = pd.merge(payment_report, product, on='product_id', how='left')

1. Check each column:

In [ ]:
#Check missing data và data type của payment_enriched và transactions
print(payment_enriched.info())
print(payment_enriched.isnull().sum())
print(transactions.info())
print(transactions.isnull().sum())

# Kiểm tra duplicates
print(payment_enriched.duplicated().sum())
print(transactions.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   report_month   919 non-null    object
 1   payment_group  919 non-null    object
 2   product_id     919 non-null    int64 
 3   source_id      919 non-null    int64 
 4   volume         919 non-null    int64 
 5   category       897 non-null    object
 6   team_own       897 non-null    object
dtypes: int64(3), object(4)
memory usage: 50.4+ KB
None
report_month      0
payment_group     0
product_id        0
source_id         0
volume            0
category         22
team_own         22
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324002 entries, 0 to 1324001
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transaction_id  1324002 non-null  int64  
 1   merchant_id     1324002 non-null  int64  


1. df payment_enriched:
- 919 rows, 7 columns
- report_month, payment_group, category, team_own: object
- product_id, source_id, volume: int64
- category, team_own: 22 null => *thay thế bằng unknown*
- Không có duplicate
2. df transactions
- 492 rows, 3 columns
- transaction_id, merchant_id, volume, transType, transStatus, timeStamp: int64
- sender_id, receiver_id: float_64 => *đổi int64*
- extra_info: object
- sender_id: 49059 giá trị null.
- receiver_id có 164795 giá trị null.
- extra_info có 1,317,907 giá trị null => gần như không có thông tin trong cột này => *no action*
- 28 duplicates => *bỏ duplicate*



In [ ]:
# Thay thế giá trị null trong cột 'category' và 'team_own' bằng 'unknown'
payment_enriched['category'] = payment_enriched['category'].fillna('unknown')
payment_enriched['team_own'] = payment_enriched['team_own'].fillna('unknown')
# Bỏ duplicates trong transactions
transactions.drop_duplicates(inplace=True)
#Thay thế giá trị null trong cột 'sender_id' và 'receiver_id' bằng -1 (không có dữ liệu)
transactions['sender_id'] = transactions['sender_id'].fillna(-1)
transactions['receiver_id'] = transactions['receiver_id'].fillna(-1)
#Chuyển kiểu dữ liệu sender_id và receiver_id từ float64 sang int64
transactions['sender_id'] = transactions['sender_id'].astype('int64')
transactions['receiver_id'] = transactions['receiver_id'].astype('int64')

2. Summarize numerical data:

In [ ]:
# Kiểm tra số liệu trong 2 df
print(payment_enriched.describe())
print(transactions.describe())

         product_id   source_id        volume
count    919.000000  919.000000  9.190000e+02
mean    1192.517954   44.875952  1.978574e+08
std     1293.463329    0.910995  8.367595e+08
min        3.000000   37.000000  5.500000e+03
25%      640.000000   45.000000  1.250000e+06
50%     1059.000000   45.000000  7.982015e+06
75%     1585.000000   45.000000  5.447599e+07
max    15067.000000   45.000000  1.383171e+10
       transaction_id   merchant_id        volume     transType   transStatus  \
count    1.323974e+06  1.323974e+06  1.323974e+06  1.323974e+06  1.323974e+06   
mean     3.002234e+09  2.460332e+03  2.388093e+05  6.979182e+00 -1.204636e+01   
std      1.042617e+07  3.304293e+03  9.681107e+05  7.459665e+00  5.577851e+01   
min      3.000000e+09  5.000000e+00  1.000000e+00  2.000000e+00 -1.333000e+03   
25%      3.001121e+09  3.050000e+02  1.000000e+04  2.000000e+00  1.000000e+00   
50%      3.002200e+09  2.250000e+03  3.000000e+04  2.000000e+00  1.000000e+00   
75%      3.003255e+

Không có incorrect values

# **Part II: Data Wrangling**

**1. Top 3 product_ids with the highest volume.**

In [ ]:
# Top 3 product_id có volume cao nhất
top_3_product_ids = payment_enriched.groupby('product_id')['volume'].sum().nlargest(3)
print(top_3_product_ids)

product_id
1976    61797583647
429     14667676567
372     13713658515
Name: volume, dtype: int64


Product_id của Top 3 sản phẩm có hiệu suất lớn nhất: 1976, 429, 372

**2**. **Given that 1 product_id is only owed by 1 team, are there any abnormal products against this rule?**

In [ ]:
# Kiểm tra sản phẩm thuộc nhiều hơn 1 team
team_check = payment_enriched.groupby('product_id')['team_own'].nunique()
abnormal_products = team_check[team_check > 1]
print(abnormal_products)

Series([], Name: team_own, dtype: int64)


=> Không có sản phẩm nào thuộc nhiều hơn 1 team

**3. Find the team has had the lowest performance (lowest volume) since Q2.2023. Find the category that contributes the least to that team.**

In [ ]:
# Lọc các dữ liệu từ Q2.2023 trở đi
payment_enriched['report_month'] = pd.to_datetime(payment_enriched['report_month'], format='%Y-%m')
q2_2023 = payment_enriched[payment_enriched['report_month'] >= '2023-04']

# Team có hiệu suất thấp nhất
team_lowest_performance = q2_2023.groupby('team_own')['volume'].sum().nsmallest(1)
print(team_lowest_performance)

# Tìm category đóng góp ít nhất trong team đó
lowest_team = team_lowest_performance.idxmin()
lowest_team_category = q2_2023[q2_2023['team_own'] == lowest_team].groupby('category')['volume'].sum().nsmallest(1)
print(lowest_team_category)

team_own
APS    51141753
Name: volume, dtype: int64
category
PXXXXXE    25232438
Name: volume, dtype: int64


- Team có hiệu suất thấp nhất: APS (volume: 51141753)
- Category đóng góp ít nhất: PXXXXXE (volume: 25232438)

**4. Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?**

In [ ]:
# Tìm source_id có đóng góp cao nhất cho giao dịch 'refund'
refunds = payment_enriched[payment_enriched['payment_group'] == 'refund']
top_refund_source = refunds.groupby('source_id')['volume'].sum().idxmax()
print(top_refund_source)

38


Source_id có bị hoàn tiền nhiều nhất là 38

**5. Define type of transactions (‘transaction_type’) for each row**

In [ ]:
# Thiết lập hàm để xác định các loại giao dịch
def classify_transaction(row):
    if row['transType'] == 2 and row['merchant_id'] == 1205:
        return 'Bank Transfer Transaction'
    elif row['transType'] == 2 and row['merchant_id'] == 2260:
        return 'Withdraw Money Transaction'
    elif row['transType'] == 2 and row['merchant_id'] == 2270:
        return 'Top Up Money Transaction'
    elif row['transType'] == 2:
        return 'Payment Transaction'
    elif row['transType'] == 8 and row['merchant_id'] == 2250:
        return 'Transfer Money Transaction'
    elif row['transType'] == 8:
        return 'Split Bill Transaction'
    else:
        return 'Invalid Transaction'

# Áp dụng hàm để thêm cột 'transaction_type'
transactions['transaction_type'] = transactions.apply(classify_transaction, axis=1)

# Kiểm tra kết quả
transactions[['transType', 'merchant_id', 'transaction_type']].head()

,transType,merchant_id,transaction_type
0,24,5,Invalid Transaction
1,2,305,Payment Transaction
2,22,7255,Invalid Transaction
3,2,2270,Top Up Money Transaction
4,2,2275,Payment Transaction


**6. Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers.**

In [ ]:
#Thiết lập hàm
def calculate_stats(group):
    number_of_transactions = len(group)  # Số lượng giao dịch
    total_volume = group['volume'].sum()  # Tổng volume
    unique_senders = group['sender_id'].nunique()  # Số lượng sender khác nhau
    unique_receivers = group['receiver_id'].nunique()  # Số lượng receiver khác nhau

    # Trả về một Series chứa các thống kê
    return pd.Series({
        'number_of_transactions': number_of_transactions,
        'total_volume': total_volume,
        'unique_senders': unique_senders,
        'unique_receivers': unique_receivers
    })
# Tính toán các thống kê bằng cách sử dụng hàm đã định nghĩa
summary_stats = transactions.groupby('transaction_type').apply(calculate_stats).reset_index()

# Kiểm tra kết quả
print(summary_stats)

             transaction_type  number_of_transactions  total_volume  \
0   Bank Transfer Transaction                   37879   50605806190   
1         Invalid Transaction                  220658   24659277889   
2         Payment Transaction                  398665   71850608441   
3      Split Bill Transaction                    1376       4901464   
4    Top Up Money Transaction                  290498  108605618829   
5  Transfer Money Transaction                  341173   37032880492   
6  Withdraw Money Transaction                   33725   23418181420   

   unique_senders  unique_receivers  
0           23156              9272  
1            2709             89717  
2          139583            113299  
3            1323               572  
4          110409            110409  
5           39021             34585  
6           24814             24814  


<ipython-input-32-de74beb1b9a1>:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_stats = transactions.groupby('transaction_type').apply(calculate_stats).reset_index()
